In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split,KFold
from sklearn import metrics
from sklearn.metrics import classification_report
from matplotlib.pyplot import figure
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras.utils import np_utils
from keras import layers
import keras_tuner
from keras_tuner.tuners import RandomSearch
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you w

### Neural Network Modeling Overview ###
In this section, I will try 3, 4, and 5 layers feed forward NN models. For each model, I will use random search to find the best hyperparameters for each model.

### Step1: Load the train and test data ###

In [2]:
train = pd.read_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Classifications NN ML Nested Random Search/data/train_preprocessed.csv")
test = pd.read_csv("/Users/haochunniu/Desktop/Kaggle Compatition/Bank Term Deposit Classifications NN ML Nested Random Search/data/test_preprocessed.csv")

y_train, y_test = train['y'], test['y']
x_train = train.drop(columns=['y'])
x_test = test.drop(columns=['y'])

### Step2: Calculate class weight for NN ###
Given that the data is extremely imbalabced, we also need to set class weight to force more focus on the minorities.

In [3]:
nn_class_weights = compute_class_weight(class_weight = 'balanced',classes = np.unique(y_train),y = y_train)
nn_class_weights = dict(zip([0,1], nn_class_weights))
print(nn_class_weights)

{0: 0.566241671258955, 1: 4.274059368500661}


### Step3: Random search on 3 layers NN model ###

In [5]:
#Random Search on 3 Layers Neural Network
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu']),
                           input_dim=x_train.shape[1]))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(1,activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values = [0.01, 0.005,0.001])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

tuner=RandomSearch(build_model,
                   objective="val_loss",
                   max_trials=5,
                   overwrite=True,
                   seed=9,
                   executions_per_trial=1)

#Keras cannot input object data type, so no matter the column is boolean or numeric we need to transform them to float32
x_train_float = np.asarray(x_train).astype(np.float32)
x_test_float = np.asarray(x_test).astype(np.float32)


tuner.search(x=x_train_float,
             y=y_train,
             epochs=500,
             batch_size=512,
             validation_data=(x_test_float,y_test),
             class_weight = nn_class_weights)

Trial 5 Complete [00h 02m 29s]
val_loss: 0.15603642165660858

Best val_loss So Far: 0.048848044127225876
Total elapsed time: 00h 15m 30s
INFO:tensorflow:Oracle triggered exit


In [6]:
# Result
result=tuner.get_best_hyperparameters()[0].values
print('The best 3 layers DNN parameters would be {} neurons, {} as activation function, and {} learning rate.'.format(result['units'],result['activation'],result['learning_rate']))
#print('------------------------------------------')
#print(tuner.results_summary())

The best 3 layers DNN parameters would be 300 neurons, relu as activation function, and 0.001 learning rate.


In [7]:
#Get the final model
Three_layers_NN=tuner.get_best_models()[0]

#Get the predicton on validation data
y_pred=Three_layers_NN.predict(x_test_float)
y_pred=np.round(y_pred)

print(classification_report(y_test,y_pred))

142/142 [==============================] - 0s 901us/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      4000
           1       0.85      0.98      0.91       521

    accuracy                           0.98      4521
   macro avg       0.92      0.98      0.95      4521
weighted avg       0.98      0.98      0.98      4521



In [8]:
#Save the final 3 layers NN model
import joblib
joblib.dump(Three_layers_NN, 'Final_3LayersNN.pkl')

#Load the model
#Three_layers_NN = joblib.load("Final_3LayersNN.pkl")

INFO:tensorflow:Assets written to: ram://02ffb0fd-026a-4301-80b6-4073e2dae4e6/assets


['Final_3LayersNN.pkl']

### Step4: Random search on 4 layers NN model ###

In [4]:
#Random Search on 4 Layers Neural Network
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu']),
                           input_dim=x_train.shape[1]))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(1,activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values = [0.01, 0.005,0.001])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

tuner=RandomSearch(build_model,
                   objective="val_loss",
                   max_trials=5,
                   overwrite=True,
                   seed=9,
                   executions_per_trial=1)

#Keras cannot input object data type, so no matter the column is boolean or numeric we need to transform them to float32
x_train_float = np.asarray(x_train).astype(np.float32)
x_test_float = np.asarray(x_test).astype(np.float32)


tuner.search(x=x_train_float,
             y=y_train,
             epochs=500,
             batch_size=512,
             validation_data=(x_test_float,y_test),
             class_weight = nn_class_weights)

Trial 5 Complete [00h 04m 03s]
val_loss: 0.15244600176811218

Best val_loss So Far: 0.03336123377084732
Total elapsed time: 00h 23m 13s
INFO:tensorflow:Oracle triggered exit


In [5]:
# Result
result=tuner.get_best_hyperparameters()[0].values
print('The best 4 layers DNN parameters would be {} neurons, {} as activation function, and {} learning rate.'.format(result['units'],result['activation'],result['learning_rate']))
#print('------------------------------------------')
#print(tuner.results_summary())

The best 4 layers DNN parameters would be 300 neurons, relu as activation function, and 0.001 learning rate.


In [6]:
#Get the final model
Four_layers_NN=tuner.get_best_models()[0]

#Get the predicton on validation data
y_pred=Four_layers_NN.predict(x_test_float)
y_pred=np.round(y_pred)

print(classification_report(y_test,y_pred))

142/142 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      4000
           1       0.88      1.00      0.93       521

    accuracy                           0.98      4521
   macro avg       0.94      0.99      0.96      4521
weighted avg       0.99      0.98      0.98      4521



In [7]:
#Save the final 4 layers NN model
import joblib
joblib.dump(Four_layers_NN, 'Final_4LayersNN.pkl')

#Load the model
#Fourlayers_NN = joblib.load("Final_4LayersNN.pkl")

INFO:tensorflow:Assets written to: ram://2adca448-7549-4dbc-bf9a-2240fefef010/assets


['Final_4LayersNN.pkl']

### Step5: Random search on 5 layers NN model ###

In [8]:
#Random Search on 5 Layers Neural Network
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu']),
                           input_dim=x_train.shape[1]))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=100,
                                        max_value=300,
                                        step=50),
                           activation=hp.Choice('activation',values = ['sigmoid','relu'])))
    model.add(layers.Dense(1,activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate', values = [0.01, 0.005,0.001])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

tuner=RandomSearch(build_model,
                   objective="val_loss",
                   max_trials=5,
                   overwrite=True,
                   seed=9,
                   executions_per_trial=1)

#Keras cannot input object data type, so no matter the column is boolean or numeric we need to transform them to float32
x_train_float = np.asarray(x_train).astype(np.float32)
x_test_float = np.asarray(x_test).astype(np.float32)


tuner.search(x=x_train_float,
             y=y_train,
             epochs=600,
             batch_size=512,
             validation_data=(x_test_float,y_test),
             class_weight = nn_class_weights)

Trial 5 Complete [00h 05m 58s]
val_loss: 0.1476597934961319

Best val_loss So Far: 0.02121235430240631
Total elapsed time: 00h 36m 47s
INFO:tensorflow:Oracle triggered exit


In [9]:
# Result
result=tuner.get_best_hyperparameters()[0].values
print('The best 5 layers DNN parameters would be {} neurons, {} as activation function, and {} learning rate.'.format(result['units'],result['activation'],result['learning_rate']))
#print('------------------------------------------')
#print(tuner.results_summary())

The best 5 layers DNN parameters would be 300 neurons, relu as activation function, and 0.001 learning rate.


In [10]:
#Get the final model
Five_layers_NN=tuner.get_best_models()[0]

#Get the predicton on validation data
y_pred=Five_layers_NN.predict(x_test_float)
y_pred=np.round(y_pred)

print(classification_report(y_test,y_pred))

142/142 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4000
           1       0.93      1.00      0.96       521

    accuracy                           0.99      4521
   macro avg       0.97      1.00      0.98      4521
weighted avg       0.99      0.99      0.99      4521



In [11]:
#Save the final 5 layers NN model
import joblib
joblib.dump(Five_layers_NN, 'Final_5LayersNN.pkl')

#Load the model
#Fivelayers_NN = joblib.load("Final_5LayersNN.pkl")

INFO:tensorflow:Assets written to: ram://47d80936-55fc-4975-ae3e-2ef1c7ba9072/assets


['Final_5LayersNN.pkl']

### Conclusion: ###
Among all machine learning models, the XGBoost model with ETA = 1.5, max_depth = 9, n_estimators = 60, and gamma = 2 is the best model. In the test data, the model could capture all postivie cases but ***the precision is quite bad with only 80%***. On the other hand, the ***5 layers NN model with 300 neurons, relu as activation function, and 0.001 learning rate*** is the best NN model. The model does a fantastic job that it not only could ***capture all postivie cases*** but also could ***boost the precision up to 93%*** in test data. Thus, the ***final 5 layers NN model*** is the best classification model.